# Recall of cause metrics

In [9]:
%reload_ext autoreload
%autoreload 2

In [10]:
SOCKSHOP_DATASET_ID = "9n6mf"

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["font.size"] = 7
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['axes.grid'] = True

In [12]:
import sys
sys.path.append('../')

In [13]:
from meltria import loader
from eval import validation

In [32]:
metrics_files = !find "/datasets/argowf-chaos-{SOCKSHOP_DATASET_ID}/" -type f -name "*.json"
records = loader.load_dataset(metrics_files, target_metric_types={
        "containers": True,
        "services": True,
        "nodes": True,
        "middlewares": True,
    },
    num_datapoints=4*45, # 45min
)

In [33]:
well_injected_records = validation.find_records_detected_anomalies_of_sli(records, faulty_datapoints=4*5)
well_injected_records = validation.find_records_detected_anomalies_of_cause_metrics(well_injected_records, faulty_datapoints=4*5)

In [34]:
len(well_injected_records)

120

In [35]:
from eval.tsdr import sweep_tsdr_for_recall

In [39]:
score_df = sweep_tsdr_for_recall(
    records=well_injected_records, 
    list_of_tsdr_options=[
        {
            "enable_unireducer": False,
            "enable_multireducer": True,
            # "step1_enable_smoother": False,
            # "step1_smoother_window_size": 4,
            # "step1_method_name": "residual_integral",
            # "step1_residual_integral_threshold": 20,
            # "step1_residual_integral_change_start_point": False,
            # "step1_residual_integral_change_start_point_n_sigma": 3,
            # "step2_enable_smoother": True,
            # "step2_smoother_window_size": 4,
            "sli_anomaly_start_time_index": 4*30,
            "step2_clustering_method_name": "changepoint",
            "step2_changepoint_n_bkps": 1,
            "step2_changepoint_proba_threshold": 0.5,
            # "step2_clustering_choice_method": "nearest_sli_changepoint",
            "step2_clustering_choice_method": "max_members_changepoint",
            # "step2_dbscan_min_pts": 1,
            # "step2_dbscan_dist_type": "pearsonr",  # 'pearsonr' or 'sbd'
            # "step2_dbscan_algorithm": "hdbscan",  # 'dbscan' or 'hdbscan'
        # "step2_clustering_series_type": "raw",  # 'raw', 'anomaly_score' or 'binary_anomaly_score'
            # "step2_clustering_choice_method": "medoid",  # 'medoid' or 'maxsum'
        },
    ],
    use_manually_selected_metrics=[False],
)

In [41]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    cols = score_df.loc[:, score_df.columns.str.contains("reduced/|recall", regex=True)].columns.tolist()
    display(score_df.groupby(["metric_types/middlewares"])[cols].agg(["mean", "max", "min"]))
    display(score_df.groupby(["chaos_type", "metric_types/middlewares"])[cols].agg(["mean", "max", "min"]))
    display(score_df.groupby(["chaos_comp", "metric_types/middlewares"])[cols].agg(["mean", "max", "min"]))
    display(score_df.set_index(["chaos_type", "chaos_comp", "chaos_case_num", "metric_types/middlewares"]).sort_index()[cols])

reduced/mand_recall            \
                                        mean  max  min   
metric_types/middlewares                                 
False                               0.743750  1.0  0.0   
True                                0.778296  1.0  0.0   

                         reduced/n_total_mand_metrics          \
                                                 mean max min   
metric_types/middlewares                                        
False                                        4.433333   6   3   
True                                         8.416667  26   3   

                         reduced/n_found_mand_metrics         reduced/recall  \
                                                 mean max min           mean   
metric_types/middlewares                                                       
False                                           3.300   6   0       0.599615   
True                                            6.575  23   0       0.673226   

                                        reduced/n_total_metrics          \
                               max  min                    mean max min   
metric_types/middlewares                                                  
False                     0.923077  0.0               10.333333  13   5   
True                      0.944444  0.0               15.566667  28   5   

                         reduced/n_found_metrics          
                                            mean max min  
metric_types/middlewares                                  
False                                   6.341667  12   0  
True                                   10.425000  23   0

reduced/mand_recall       \
                                                            mean  max   
chaos_type          metric_types/middlewares                            
pod-cpu-hog         False                               0.787037  1.0   
                    True                                0.851852  1.0   
pod-memory-hog      False                               0.765152  1.0   
                    True                                0.741391  1.0   
pod-network-latency False                               0.681250  1.0   
                    True                                0.752692  1.0   

                                                        \
                                                   min   
chaos_type          metric_types/middlewares             
pod-cpu-hog         False                     0.333333   
                    True                      0.166667   
pod-memory-hog      False                     0.000000   
                    True                      0.000000   
pod-network-latency False                     0.000000   
                    True                      0.000000   

                                             reduced/n_total_mand_metrics      \
                                                                     mean max   
chaos_type          metric_types/middlewares                                    
pod-cpu-hog         False                                        3.000000   3   
                    True                                         5.250000   8   
pod-memory-hog      False                                        6.000000   6   
                    True                                         8.613636  11   
pod-network-latency False                                        4.000000   4   
                    True                                        11.050000  26   

                                                 reduced/n_found_mand_metrics  \
                                             min                         mean   
chaos_type          metric_types/middlewares                                    
pod-cpu-hog         False                      3                     2.361111   
                    True                       3                     4.500000   
pod-memory-hog      False                      6                     4.590909   
                    True                       6                     6.113636   
pod-network-latency False                      4                     2.725000   
                    True                       4                     8.950000   

                                                     reduced/recall            \
                                             max min           mean       max   
chaos_type          metric_types/middlewares                                    
pod-cpu-hog         False                      3   1       0.692308  0.923077   
                    True                       8   1       0.767643  0.944444   
pod-memory-hog      False                      6   0       0.573427  0.769231   
                    True                      11   0       0.601785  0.851852   
pod-network-latency False                      4   0       0.545000  0.800000   
                    True                      23   0       0.666835  0.909091   

                                                        \
                                                   min   
chaos_type          metric_types/middlewares             
pod-cpu-hog         False                     0.384615   
                    True                      0.312500   
pod-memory-hog      False                     0.076923   
                    True                      0.071429   
pod-network-latency False                     0.000000   
                    True                      0.000000   

                                             reduced/n_total_metrics          \
                                                                mean ma

reduced/mand_recall                      \
                                                     mean       max       min   
chaos_comp   metric_types/middlewares                                           
carts        False                               0.942308  1.000000  0.666667   
             True                                0.819706  1.000000  0.636364   
carts-db     False                               0.688889  1.000000  0.000000   
             True                                0.685758  1.000000  0.000000   
catalogue    False                               0.855556  1.000000  0.000000   
             True                                0.855556  1.000000  0.000000   
catalogue-db False                               0.641026  1.000000  0.000000   
             True                                0.782051  1.000000  0.250000   
orders       False                               0.694444  1.000000  0.000000   
             True                                0.758838  0.909091  0.166667   
orders-db    False                               0.430556  1.000000  0.000000   
             True                                0.740720  1.000000  0.454545   
payment      False                               0.560606  1.000000  0.000000   
             True                                0.560606  1.000000  0.000000   
user         False                               0.844444  1.000000  0.333333   
             True                                0.844444  1.000000  0.333333   
user-db      False                               0.940476  1.000000  0.333333   
             True                                0.901786  1.000000  0.700000   

                                      reduced/n_total_mand_metrics          \
                                                              mean max min   
chaos_comp   metric_types/middlewares                                        
carts        False                                        4.538462   6   3   
             True                                        15.615385  26   6   
carts-db     False                                        4.333333   6   3   
             True                                         9.666667  11   8   
catalogue    False                                        4.333333   6   3   
             True                                         4.333333   6   3   
catalogue-db False                                        4.461538   6   3   
             True                                         4.461538   6   3   
orders       False                                        4.333333   6   3   
             True                                        14.333333  26   6   
orders-db    False                                        4.583333   6   3   
             True                                         9.916667  11   8   
payment      False                                        4.636364   6   3   
             True                                         4.636364   6   3   
user         False                                        4.333333   6   3   
             True                                         4.333333   6   3   
user-db      False                                        4.428571   6   3   
             True                                         9.428571  10   8   

                                      reduced/n_found_mand_metrics          \
                                                              mean max min   
chaos_comp   metric_types/middlewares                                        
carts        False                                        4.307692   6   2   
             True                                        12.384615  21   5   
carts-db     False                                        2.666667   4   0   
             True                                         6.266667  10   0   
catalogue    False                                        3.733333   6   0   
             True                                         3.733333   6   

KeyError: "None of ['chaos_type', 'chaos_comp', 'chaos_case_num', 'metric_types/middlewares'] are in the columns"